In [1]:
import findspark
findspark.init()

import pyspark.sql.functions as F

from etl import SparkETL

In [2]:
etl = SparkETL()
spark = etl.get_spark()

22/05/09 11:23:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/09 11:23:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/09 11:23:10 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [18]:
immigration = etl.read_clean_table('immigration')

In [19]:
time_dim = etl.read_dim_table('time_dim')

In [20]:
def time_dim_nk(df):
    return (
        df
        .select(
            'arrival_date'
        )
        .distinct()
    )

In [21]:
def join_immigration_time_dim(nk1, nk2):
    return (
        nk1
        .join(
            nk2,
            on=(
                (nk1['arrival_date'] == nk2['date'])
              ),
            how='leftanti'
        )
    )

In [40]:
def fill_time_dim(df):
    return df.select(
        F.monotonically_increasing_id().alias('time_id'),
        F.col('arrival_date').alias('date'),
        F.expr('YEAR(arrival_date)').alias('year'),
        F.expr('MONTH(arrival_date)').alias('month_id'),
        F.expr('DAY(arrival_date)').alias('day'),
        F.expr('WEEKDAY(arrival_date)').alias('weekday_id'),
        F.expr("DATE_FORMAT(arrival_date, 'MMM')").alias('month'),
        F.expr("DATE_FORMAT(arrival_date, 'E')").alias('weekday'),
        F.expr("DATE_FORMAT(arrival_date, 'E') IN ('Sat', 'Sun')").alias('weekend')
    )

In [23]:
etl.read_dim_table('time_dim').toPandas()

,time_id,date,year,month_id,day,weekday_id,month,weekday,weekend


In [44]:
time_dim_append = (
    immigration
    .pipe(time_dim_nk)
    .pipe(join_immigration_time_dim, time_dim)
    .pipe(fill_time_dim)
)

In [45]:
etl.save_dim_table(time_dim_append, 'time_dim')

22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 63.33% for 12 writers
22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 69.09% for 11 writers
22/05/09 11:45:41 WARN MemoryManager: Total allocation exceeds 95.

In [46]:
etl.read_dim_table('time_dim').sort('day').toPandas()


,time_id,date,year,month_id,day,weekday_id,month,weekday,weekend
0,953482739712,2016-04-01,2016,4,1,4,Apr,Fri,False
1,635655159808,2016-04-02,2016,4,2,5,Apr,Sat,True
2,1065151889408,2016-04-03,2016,4,3,6,Apr,Sun,True
3,377957122048,2016-04-04,2016,4,4,0,Apr,Mon,False
4,1357209665536,2016-04-05,2016,4,5,1,Apr,Tue,False
5,1374389534720,2016-04-06,2016,4,6,2,Apr,Wed,False
6,979252543488,2016-04-07,2016,4,7,3,Apr,Thu,False
7,592705486848,2016-04-08,2016,4,8,4,Apr,Fri,False
8,824633720832,2016-04-09,2016,4,9,5,Apr,Sat,True
9,1597727834112,2016-04-10,2016,4,10,6,Apr,Sun,True
